## Spanish High Speed Rail Tickets Pricing

This is a dataset from [the Gurus](https://thegurus.tech/). The data is scraped in real time and is used for the purpose of enlightening the customers with the price that is best for the given purchase. 

Tickets pricing can change with the demand and time. Users can set up some alarm/remainder (email) when the prices drop below a mean price. 

In [1]:
# Defining packages
import pandas as pd

In [2]:
df = pd.read_csv('renfe.csv')
del df['Unnamed: 0'] # Deleting the unnamed column

In [3]:
# Let's see the data in hand
df.head()

,insert_date,origin,destination,start_date,end_date,train_type,price,train_class,fare
0,2019-04-19 05:31:43,MADRID,SEVILLA,2019-05-29 06:20:00,2019-05-29 09:16:00,AV City,38.55,Turista,Promo
1,2019-04-19 05:31:43,MADRID,SEVILLA,2019-05-29 07:00:00,2019-05-29 09:32:00,AVE,53.40,Turista,Promo
2,2019-04-19 05:31:43,MADRID,SEVILLA,2019-05-29 07:30:00,2019-05-29 09:51:00,AVE,47.30,Turista,Promo
3,2019-04-19 05:31:43,MADRID,SEVILLA,2019-05-29 08:00:00,2019-05-29 10:32:00,AVE,69.40,Preferente,Promo
4,2019-04-19 05:31:43,MADRID,SEVILLA,2019-05-29 08:30:00,2019-05-29 11:14:00,ALVIA,NaN,Turista,Promo


As it can be seen there are 9 columns with date when the data was inserted (`insert_date`), origin, destination, start and end date of journey, type of the train, price, train class, fare type. 

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2579771 entries, 0 to 2579770
Data columns (total 9 columns):
insert_date    object
origin         object
destination    object
start_date     object
end_date       object
train_type     object
price          float64
train_class    object
fare           object
dtypes: float64(1), object(8)
memory usage: 177.1+ MB


Since the columns are of datetime type. I can convert them using a function and change the type. 

In [5]:
def convert_date(df):
    return pd.to_datetime(df)

In [6]:
df.insert_date = convert_date(df.insert_date)
df.start_date = convert_date(df.start_date)
df.end_date = convert_date(df.end_date)

In [7]:
# Coversion to date - DONE!
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2579771 entries, 0 to 2579770
Data columns (total 9 columns):
insert_date    datetime64[ns]
origin         object
destination    object
start_date     datetime64[ns]
end_date       datetime64[ns]
train_type     object
price          float64
train_class    object
fare           object
dtypes: datetime64[ns](3), float64(1), object(5)
memory usage: 177.1+ MB


Converted the objects to datetime objects. This will help in better analysis

In [8]:
col_names = ['origin', 'destination', 'train_type', 'train_class', 'fare']
index_list = [1, 2, 5, 7, 8]
for col, item in zip(col_names, index_list):
    print(f'Unique Values of column {col}: {df.iloc[:,item].unique()}')

Unique Values of column origin: ['MADRID' 'SEVILLA' 'PONFERRADA' 'BARCELONA' 'VALENCIA']
Unique Values of column destination: ['SEVILLA' 'MADRID' 'VALENCIA' 'BARCELONA' 'PONFERRADA']
Unique Values of column train_type: ['AV City' 'AVE' 'ALVIA' 'INTERCITY' 'MD-LD' 'MD' 'LD-MD' 'LD' 'MD-AVE'
 'TRENHOTEL' 'AVE-LD' 'REGIONAL' 'AVE-MD' 'R. EXPRES' 'AVE-TGV' 'LD-AVE']
Unique Values of column train_class: ['Turista' 'Preferente' 'Turista con enlace' 'Turista Plus' nan
 'Cama G. Clase' 'Cama Turista']
Unique Values of column fare: ['Promo' 'Flexible' 'Promo +' nan 'Individual-Flexible' 'Adulto ida'
 'Mesa' 'Grupos Ida']


It can be seen that there are a set of unique values in origin and destination are the same. There are to and fro trains from any point A in origin to another point B in destination. What will be the unique combination available when we combine the origina and destination? Let's see. 

In [9]:
df['origin_dest'] = df.origin + '-' + df.destination
df.head(2)

,insert_date,origin,destination,start_date,end_date,train_type,price,train_class,fare,origin_dest
0,2019-04-19 05:31:43,MADRID,SEVILLA,2019-05-29 06:20:00,2019-05-29 09:16:00,AV City,38.55,Turista,Promo,MADRID-SEVILLA
1,2019-04-19 05:31:43,MADRID,SEVILLA,2019-05-29 07:00:00,2019-05-29 09:32:00,AVE,53.40,Turista,Promo,MADRID-SEVILLA


Perfect, now to test the unique possibilities of the number of trips made between an origin and destination.

In [10]:
print(f'Unique Value of origin-destination: {df.origin_dest.unique()}')

Unique Value of origin-destination: ['MADRID-SEVILLA' 'SEVILLA-MADRID' 'MADRID-VALENCIA' 'PONFERRADA-MADRID'
 'BARCELONA-MADRID' 'MADRID-BARCELONA' 'MADRID-PONFERRADA'
 'VALENCIA-MADRID']


Looks like MADRID is a key station and people travel to and from MADRID to all other places. 

### Handling Missing values (Numeric columns) - 1
Let's check for the missing values in the price column. 

In [11]:
print(f'Number of missing values in price: {sum(df.price.isnull())}')
print(f'Fraction of missing values: {100*sum(df.price.isnull())/ df.iloc[:,0].count()} %')

Number of missing values in price: 310681
Fraction of missing values: 12.04296815492538 %


We cannot touch this feature. It is the target column and should not be disturbed. Better to remove the Non numeric columns in the dataframe. That's what we do next.

### Handling Missing values (Non-integer column)
There are missing values in the column `train_class`. This is seen as a NaN value. Let's dig more to see how many are missing

In [12]:
# Train class type
print(f'Number of missing values in train_class: {sum(df.train_class.isnull())}')
print(f'Fraction of missing values: {100*sum(df.train_class.isnull())/ df.iloc[:,0].count()} %')

# Fare type 
print(f'Number of missing values in fare: {sum(df.fare.isnull())}')
print(f'Fraction of missing values: {100*sum(df.fare.isnull())/ df.iloc[:,0].count()} %')

Number of missing values in train_class: 9664
Fraction of missing values: 0.37460689340255393 %
Number of missing values in fare: 9664
Fraction of missing values: 0.37460689340255393 %


Since only $0.3\%$ is missing. It is better to remove the observations that tend to provide trouble in analysis. Fingers crossed on how many rows with missing data in pricing columns is missed. 

In [13]:
# Removing the rows that contain NAN's in train_class
df.dropna(how='any',axis='rows',subset=['train_class', 'fare'], inplace = True)
# Train class type
print(f'Unique Values of column train_class: {df.train_class.unique()}')
print(f'Number of missing values in train_class: {sum(df.train_class.isnull())}')

# Fare type
print(f'Unique Values of column fare: {df.fare.unique()}')
print(f'Number of missing values in fare: {sum(df.fare.isnull())}')

Unique Values of column train_class: ['Turista' 'Preferente' 'Turista con enlace' 'Turista Plus'
 'Cama G. Clase' 'Cama Turista']
Number of missing values in train_class: 0
Unique Values of column fare: ['Promo' 'Flexible' 'Promo +' 'Individual-Flexible' 'Adulto ida' 'Mesa'
 'Grupos Ida']
Number of missing values in fare: 0


### Handling Missing values (Numeric columns) - 2
Let's check for the missing values in the price column after the removal of the non-numeric NAN's.

In [14]:
print(f'Number of missing values in price: {sum(df.price.isnull())}')
print(f'Fraction of missing values: {100*sum(df.price.isnull())/ df.iloc[:,0].count()} %')

Number of missing values in price: 301017
Fraction of missing values: 11.712236105345031 %


We cannot fill the target variable column with mean/ median. It is not logical to do so. Since the number of missing values is well above $10\%$, it is better to not impute them and use tree-based models for prediction. This value was $12\%$ before removing the rows in the `fare` and `train_class` columns. Still, it's better to use this instead of completely removing the rows that has no price values.

### Feature Engineering

Let's define a new column which gives the time travelled by a passenger from one place to another. 

In [15]:
df['travel_time'] = df.end_date - df.start_date
df.head(2)

,insert_date,origin,destination,start_date,end_date,train_type,price,train_class,fare,origin_dest,travel_time
0,2019-04-19 05:31:43,MADRID,SEVILLA,2019-05-29 06:20:00,2019-05-29 09:16:00,AV City,38.55,Turista,Promo,MADRID-SEVILLA,02:56:00
1,2019-04-19 05:31:43,MADRID,SEVILLA,2019-05-29 07:00:00,2019-05-29 09:32:00,AVE,53.40,Turista,Promo,MADRID-SEVILLA,02:32:00


In [16]:
def convert_to_seconds(time):
    return int(time.total_seconds())
df.travel_time = df.travel_time.apply(convert_to_seconds)
df.head(2)

,insert_date,origin,destination,start_date,end_date,train_type,price,train_class,fare,origin_dest,travel_time
0,2019-04-19 05:31:43,MADRID,SEVILLA,2019-05-29 06:20:00,2019-05-29 09:16:00,AV City,38.55,Turista,Promo,MADRID-SEVILLA,10560
1,2019-04-19 05:31:43,MADRID,SEVILLA,2019-05-29 07:00:00,2019-05-29 09:32:00,AVE,53.40,Turista,Promo,MADRID-SEVILLA,9120


Since the columns `start_date` and `end_date` are converted to `travel_time`. We can remove them along with the `insert_date` column for analysis purposes.

In [17]:
df = df[['origin_dest', 'train_type', 'train_class', 'fare', 'travel_time', 'price']]
df.head(2)

,origin_dest,train_type,train_class,fare,travel_time,price
0,MADRID-SEVILLA,AV City,Turista,Promo,10560,38.55
1,MADRID-SEVILLA,AVE,Turista,Promo,9120,53.40


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2570107 entries, 0 to 2579770
Data columns (total 6 columns):
origin_dest    object
train_type     object
train_class    object
fare           object
travel_time    int64
price          float64
dtypes: float64(1), int64(1), object(4)
memory usage: 137.3+ MB


In [19]:
col_names = list(df.columns[:-1])
index_list = range(5)
for col, item in zip(col_names, index_list):
    print('----------------------------------------------------------')
    print(f'Unique Values of column {col}: {df.iloc[:,item].unique()}')
    print(f'Count of Missing Values of column {col}: {sum(df.iloc[:,item].isnull())}')

----------------------------------------------------------
Unique Values of column origin_dest: ['MADRID-SEVILLA' 'SEVILLA-MADRID' 'MADRID-VALENCIA' 'PONFERRADA-MADRID'
 'BARCELONA-MADRID' 'MADRID-BARCELONA' 'MADRID-PONFERRADA'
 'VALENCIA-MADRID']
Count of Missing Values of column origin_dest: 0
----------------------------------------------------------
Unique Values of column train_type: ['AV City' 'AVE' 'ALVIA' 'INTERCITY' 'MD-LD' 'MD' 'LD-MD' 'LD' 'MD-AVE'
 'TRENHOTEL' 'AVE-LD' 'REGIONAL' 'AVE-MD' 'R. EXPRES' 'AVE-TGV' 'LD-AVE']
Count of Missing Values of column train_type: 0
----------------------------------------------------------
Unique Values of column train_class: ['Turista' 'Preferente' 'Turista con enlace' 'Turista Plus'
 'Cama G. Clase' 'Cama Turista']
Count of Missing Values of column train_class: 0
----------------------------------------------------------
Unique Values of column fare: ['Promo' 'Flexible' 'Promo +' 'Individual-Flexible' 'Adulto ida' 'Mesa'
 'Grupos Ida']


### One hot encoding the data
Let's one hot encode the object columns and the int64 `travel_time` column.

In [20]:
df = pd.get_dummies(df, columns = list(df.columns[:-2]))
df.head(3)

,travel_time,price,origin_dest_BARCELONA-MADRID,origin_dest_MADRID-BARCELONA,origin_dest_MADRID-PONFERRADA,origin_dest_MADRID-SEVILLA,origin_dest_MADRID-VALENCIA,origin_dest_PONFERRADA-MADRID,origin_dest_SEVILLA-MADRID,origin_dest_VALENCIA-MADRID,...,train_class_Turista,train_class_Turista Plus,train_class_Turista con enlace,fare_Adulto ida,fare_Flexible,fare_Grupos Ida,fare_Individual-Flexible,fare_Mesa,fare_Promo,fare_Promo +
0,10560,38.55,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
1,9120,53.40,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,8460,47.30,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0


In [21]:
# let's reorder the columns 
col_names = df.columns.to_list()
col_names.append(col_names.pop(0))
col_names.append(col_names.pop(0))
df = df[col_names]
df.head(3)

,origin_dest_BARCELONA-MADRID,origin_dest_MADRID-BARCELONA,origin_dest_MADRID-PONFERRADA,origin_dest_MADRID-SEVILLA,origin_dest_MADRID-VALENCIA,origin_dest_PONFERRADA-MADRID,origin_dest_SEVILLA-MADRID,origin_dest_VALENCIA-MADRID,train_type_ALVIA,train_type_AV City,...,train_class_Turista con enlace,fare_Adulto ida,fare_Flexible,fare_Grupos Ida,fare_Individual-Flexible,fare_Mesa,fare_Promo,fare_Promo +,travel_time,price
0,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,10560,38.55
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,9120,53.40
2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,8460,47.30


### Storing the Clean data
The data is now ohe'd and clean. Let's store it in a csv file.  

In [22]:
df.to_csv('renfe-clean.csv')

### Building a ML model 

In [23]:
import numpy as np
df = pd.read_csv('renfe-clean.csv')
del df['Unnamed: 0']
df.fillna(value=np.mean(df.price), axis = 'rows', inplace = True)

In [24]:
X, Y = df.values[:,:-1], df.values[:,-1]
Y = Y.reshape((Y.shape[0],1))
print(X.shape)
print(Y.shape)

(2570107, 38)
(2570107, 1)


In [25]:
from sklearn.model_selection import train_test_split
# For reproducibility use random_state as the universal number
X_train, X_test, y_train, y_test = train_test_split( 
          X, Y, test_size = 0.3, random_state = 42)

In [26]:
# Function to perform training with MSE
def train_using_mse(X_train, X_test, y_train): 
  
    # Creating the classifier object 
    clf_mse = DecisionTreeRegressor(criterion = "mse", 
            random_state = 42,max_depth=20, min_samples_leaf=4) 
  
    # Performing training 
    clf_mse.fit(X_train, y_train) 
    return clf_mse

### Operational Phase
This is the effect of applying the decision trees on the Data. Let's train using MSE and MAE criterion.

In [27]:
from sklearn.tree import DecisionTreeRegressor
print("Results Using MSE:")
clf_mse = train_using_mse(X_train, X_test, y_train)
print(clf_mse)

Results Using MSE:
DecisionTreeRegressor(criterion='mse', max_depth=20, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=4,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=42, splitter='best')


Thus, the trained models are fit for the given data, the next step is to evaluate the model using some testing data. This can be done using the prediction on the validation test data. 

In [28]:
from sklearn.metrics import accuracy_score
# Function to make predictions 
def prediction(X_test, clf_object): 
    # Predicton on test with MSE/ MAE
    y_pred = clf_object.predict(X_test) 
    return y_pred

# Function to calculate accuracy 
def cal_accuracy(X_test, y_test, clf_object):
    print (f"{clf_object.score(X_test,y_test)*100} %")

In [29]:
print('Prediction using MSE Regressor')
y_pred_mse = prediction(X_test, clf_mse)
print(y_pred_mse)
print("R^2 coefficient Accuracy:",end=" ")
cal_accuracy(X_test, y_test, clf_mse)

Prediction using MSE Regressor
[74.79453426 71.75426324 42.52464513 ... 28.35       63.38550342
 73.07278811]
R^2 coefficient Accuracy: 82.83892203587486 %


This model on the training set gives an accuracy of $82\%$ which is not pretty bad. So, the real deal in this dataset can be seen when the test data is present. Then the model can be evaluated in a better way. 